# PyTorchを用いたCNNによる画像認識 (データセットの作成・Data Augumentation)


---
## 目的
CIFAR10 Datasetを用いて10クラスの物体認識を行う．プログラムの構成は，MNISTによる文字認識のプログラムと同様になっているため，基礎的な説明はそちらを参照して頂きたい．このページでは，MNISTによる文字認識のプログラムとの差分について書いていく．

GPUを用いたネットワークの計算を行う．
また，Data Augmentationを用いた学習の効果について確認する．


## 準備

### Google Colaboratoryの設定確認・変更
本チュートリアルではPyTorchを利用してニューラルネットワークの実装を確認，学習および評価を行います．
**GPUを用いて処理を行うために，上部のメニューバーの「ランタイム」→「ランタイムのタイプを変更」からハードウェアアクセラレータをGPUにしてください．**


## 使用するデータセット

### データセット
今回の物体認識では，CIFAR10データセットを用いる．CIFAR10データセットは，飛行機や犬などの10クラスの物体が表示されている画像から構成されたデータセットである．

![CIFAR10_sample.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/176458/b6b43478-c85f-9211-7bc6-227d9b387af5.png)

## モジュールのインポート
はじめに必要なモジュールをインポートする．

### GPUの確認
GPUを使用した計算が可能かどうかを確認します．

`GPU availability: True`と表示されれば，GPUを使用した計算をPyTorchで行うことが可能です．
Falseとなっている場合は，上記の「Google Colaboratoryの設定確認・変更」に記載している手順にしたがって，設定を変更した後に，モジュールのインポートから始めてください．


In [ ]:
# モジュールのインポート
import os
from time import time
import numpy as np
import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms

import torchsummary

import cv2
import pickle
import urllib.request
import tarfile
from random import randint

# GPUの確認
use_cuda = torch.cuda.is_available()
print('Use CUDA:', use_cuda)

## データセットの読み込みとData Augmentation

学習データ（CIFAR10データセット）を使用するためのデータセットクラスを作成します．
まず，`MyCIFAR10`というクラスを作成します．
この際，`torch.utils.data.Dataset`クラスを継承します．

### 1. \_\_init\_\_()
`self.__init__(self, ...)`でCIFAR-10データセットのダウンロードと読み込みを行います．
まず，`urllib`を用いてwebからCIFAR10データをダウンロードします．
その後，ダウンロードした圧縮ファイルを`tarfile`を用いて解凍します．
※ 解凍したファイルを左側の「ファイル」一覧から確認して見ましょう．

次に，用意するデータが学習用データか評価用データかを指定し，読み込むファイル名を`download_list`に格納します．

CIFAR10データを読み込みます．
解凍したフォルダ内にあるデータは，pickle形式となっており，`pickle`モジュールを用いて展開・読み込みを行います．

### 2. \_\_len\_\_()
`__len__(self)`ではデータセットのサンプル数を返すように定義します．
今回は，`self.data`に格納されている画像データの枚数を返す様に定義します．
（学習用データでは50,000枚，評価用データは10,000枚）

### 3. \_\_getitem\_\_()
`__getitem__(self, item)`では，`item`で指定した番号のサンプルを読み込んで返すように定義を行います．
まず，`item`番目の画像データと対応するラベルを取得します．

次に，`self.train`が`True`の場合は，data augmentationを適用させます．
ここでは，`MyCIFAR10`クラス内の`_random_crop`と`_random_horizontal_flip`メソッドを使用し，ランダムに画像の切り取りと左右反転を適用します．

その後，画像データの画素値を0~1の範囲の値になる様に正規化を行い，，画像データの配列を`[channel, height, width]`となる様に配列操作を行い，画像データとラベルを返します．

### 4. \_random_crop(), \_random_horizontal_flip()
`_random_crop`と`_random_horizontal_flip`では，メソッドに入力された画像データに対して，それぞれ，ランダムな画像の切り取りと左右反転のdata augmentationを適用します．


In [ ]:
class MyCIFAR10(torch.utils.data.Dataset):
    base_folder = 'cifar-10-batches-py'
    url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
    filename = "cifar-10-python.tar.gz"
    train_list = [
        ['data_batch_1', 'c99cafc152244af753f735de768cd75f'],
        ['data_batch_2', 'd4bba439e000b95fd0a9bffe97cbabec'],
        ['data_batch_3', '54ebc095f3ab1f0389bbae665268c751'],
        ['data_batch_4', '634d18415352ddfa80567beed471001a'],
        ['data_batch_5', '482c414d41f54cd18b22e5b47cb7c3cb'],
    ]
    test_list = [
        ['test_batch', '40351d587109b95175f43aff81a1287e'],
    ]

    def __init__(self, root, train=True, download=True):
        super().__init__()

        self.root = root
        self.train = train
        self.download = download

        # CIFAR10データのダウンロード
        if download:
            urllib.request.urlretrieve(self.url, os.path.join(self.root, self.filename))
            with tarfile.open(os.path.join(self.root, self.filename), 'r') as tar:
                tar.extractall(path=self.root)

        # 学習，評価データの判定
        if self.train:
            downloaded_list = self.train_list
        else:
            downloaded_list = self.test_list

        # データの読み込み
        self.data = []
        self.targets = []

        for file_name, checksum in downloaded_list:
            file_path = os.path.join(self.root, self.base_folder, file_name)
            with open(file_path, 'rb') as f:
                entry = pickle.load(f, encoding='latin1')
                self.data.append(entry['data'])
                if 'labels' in entry:
                    self.targets.extend(entry['labels'])
                else:
                    self.targets.extend(entry['fine_labels'])

        # リスト形式で保存された画像データをnumpy.arrayに変換
        self.data = np.vstack(self.data).reshape(-1, 3, 32, 32)
        self.data = self.data.transpose((0, 2, 3, 1))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, item):
        img, target = self.data[item], self.targets[item]
        
        # data augmentation
        if self.train:
            img = self._random_crop(img)
            img = self._random_horizontal_flip(img)
        
        # データの正規化（0~255）
        img = img.astype(np.float32) / 255.

        # 画像の配列を入れ替え
        img = img.transpose(2, 0, 1)

        return img, target

    @staticmethod
    def _random_crop(image, min_size=24):
        crop_size = randint(24, 32)
        
        if crop_size == 32:
            return image
        else:
            top = randint(0, 32 - crop_size)
            left = randint(0, 32 - crop_size)
            image = image[left:left+crop_size, top:top+crop_size, :]
            image = cv2.resize(image, (32, 32))
            return image

    @staticmethod
    def _random_horizontal_flip(image):
        if randint(0, 1):
            image = np.flip(image, axis=0)
        return image

上で定義したデータセットクラスを用いてCIFAR10データセットを読み込みます．

また，読み込んだデータセットクラスの情報を表示します．
まず，各データセットが保有しているサンプル数を表示します．
データセットクラスに`len()`を適用すると，上で定義した`__len__()`メソッドが呼ばれ，サンプル数を返します．

次に，`train_data`のとある1サンプルを読み込みます．
`train_data[10]`とすることで，上で定義した`__getitem__()`メソッドが呼ばれ，引数の`item`に`10`が与えられ，10番目のサンプルを返します．

In [ ]:
train_data = MyCIFAR10(root="./", train=True, download=True)
test_data = MyCIFAR10(root="./", train=False, download=True)

# サンプル数の表示
print(len(train_data))
print(len(test_data))

# とあるサンプルの読み込み
img, lab = train_data[10]
print(img, lab)

## ネットワークモデルの定義
畳み込みニューラルネットワークを定義します．

ここでは，畳み込み層２層，全結合層３層から構成されるネットワークとします．

１層目の畳み込み層は入力チャンネル数が１，出力する特徴マップ数が16，畳み込むフィルタサイズが3x3です．２層目の畳み込み層は入力チャネル数が16．出力する特徴マップ数が32，畳み込むフィルタサイズは同じく3x3です．１つ目の全結合層は入力ユニット数は不定とし，出力は1024としています．次の全結合層入力，出力共に1024，出力層は入力が1024，出力が10です．これらの各層の構成を`__init__`関数で定義します．

次に，`forward`関数では，定義した層を接続して処理するように記述します．`forward`関数の引数xは入力データです．それを`__init__`関数で定義したconv1に与え，その出力を活性化関数であるrelu関数に与えます．そして，その出力をmax_pooling_2dに与えて，プーリング処理結果をhとして出力します．hはconv2に与えられて畳み込み処理とプーリング処理を行います．そして，出力hをl1に与えて全結合層の処理を行います．最終的にl3の全結合層の処理を行った出力hを戻り値としています．

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.l1 = nn.Linear(8 * 8 * 32, 1024)
        self.l2 = nn.Linear(1024, 1024)
        self.l3 = nn.Linear(1024, 10)
    
    def forward(self, x):
        h = self.pool(self.relu(self.conv1(x)))
        h = self.pool(self.relu(self.conv2(h)))
        h = h.view(h.size()[0], -1)
        h = self.relu(self.l1(h))
        h = self.relu(self.l2(h))
        h = self.l3(h)
        return h

## ネットワークの作成
上のプログラムで定義したネットワークを作成します．

CNNクラスを呼び出して，ネットワークモデルを定義します． また，GPUを使う場合（use_cuda == True）には，ネットワークモデルをGPUメモリ上に配置します． これにより，GPUを用いた演算が可能となります．

学習を行う際の最適化方法としてモーメンタムSGD(モーメンタム付き確率的勾配降下法）を利用します． また，学習率を0.01，モーメンタムを0.9として引数に与えます．

最後に，定義したネットワークの詳細情報を`torchsummary.summary()`関数を用いて表示します．


In [ ]:
model = CNN()
if use_cuda:
    model.cuda()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# モデルの情報を表示
torchsummary.summary(model, (3, 32, 32))

## 学習
読み込んだMNISTデータセットと作成したネットワークを用いて，学習を行います．

1回の誤差を算出するデータ数（ミニバッチサイズ）を64，学習エポック数を10とします．

次にデータローダーを定義します．
データローダーでは，上で読み込んだデータセット（`train_data`）を用いて，for文で指定したミニバッチサイズでデータを読み込むオブジェクトを作成します．
この時，`shuffle=True`と設定することで，読み込むデータを毎回ランダムに指定します．

次に，誤差関数を設定します．
今回は，分類問題をあつかうため，クロスエントロピー誤差を計算するための`CrossEntropyLoss`を`criterion`として定義します．

学習を開始します．

各更新において，学習用データと教師データをそれぞれ`image`と`label`とします．
学習モデルにimageを与えて各クラスの確率yを取得します．
各クラスの確率yと教師ラベルtとの誤差を`criterion`で算出します．
また，認識精度も算出します．
そして，誤差をbackward関数で逆伝播し，ネットワークの更新を行います．

In [ ]:
# ミニバッチサイズ・エポック数の設定
batch_size = 64
epoch_num = 10
n_iter = len(train_data) / batch_size

# データローダーの設定
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

# 誤差関数の設定
criterion = nn.CrossEntropyLoss()
if use_cuda:
    criterion.cuda()

# ネットワークを学習モードへ変更
model.train()

start = time()
for epoch in range(1, epoch_num+1):
    sum_loss = 0.0
    count = 0
    
    for image, label in train_loader:
        
        if use_cuda:
            image = image.cuda()
            label = label.cuda()

        y = model(image)

        loss = criterion(y, label)
        
        model.zero_grad()
        loss.backward()
        optimizer.step()
        
        sum_loss += loss.item()
        
        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)
        
    print("epoch: {}, mean loss: {}, mean accuracy: {}, elapsed_time :{}".format(epoch,
                                                                                 sum_loss / n_iter,
                                                                                 count.item() / len(train_loader),
                                                                                 time() - start))

## テスト
学習したネットワークモデルを用いて評価を行います．

In [ ]:
# データローダーの準備
test_loader = torch.utils.data.DataLoader(test_data, batch_size=100, shuffle=False)

# ネットワークを評価モードへ変更
model.eval()

# 評価の実行
count = 0
with torch.no_grad():
    for image, label in test_loader:

        if use_cuda:
            image = image.cuda()
            label = label.cuda()
            
        y = model(image)

        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)

print("test accuracy: {}".format(count.item() / 10000.))

## torchvision.datasetsクラス

CIFAR10などの一般的なデータセットは`torchvision.datasets`にすでに用意されています．
以下では，`torchvision.datasets`に用意されているCIFAR10データセットを使用した場合のデータセットおよび，data augmentationの準備方法を紹介します．

はじめに`torchvision`の`transform`を用いて，各サンプルを読み込む際の前処理を定義します．
`transform`には，上で定義した様な左右反転やランダムな切り取りなどの関数が用意されています．
これらを`transforms.Compose()`によって，まとめて定義することで，一連の処理としてまとめて定義することができます．
以下の学習データ用の`transform_train`では，
1. ランダムな画像の切り取り
2. ランダムな左右反転
3. 画像データをtorch.Tensorのオブジェクトへ変換
という処理を定義しています．

つぎに，`torchvision.datasets.CIFAR10`クラスを使用して，CIFAR10データセットを準備します．
この時，`transform`の引数に上で定義した処理を指定することで，各サンプルを読み込む際に，指定した処理を行った上でデータを返すことが可能です．

これらのデータセットクラスを上で行った学習・評価プログラムに使用することで，同様の学習・評価を行うことが可能です．

In [ ]:
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=1),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor()])
transform_test = transforms.Compose([transforms.ToTensor()])

train_data = torchvision.datasets.CIFAR10(root="./", train=True, transform=transform_train, download=True)
test_data = torchvision.datasets.CIFAR10(root="./", train=False, transform=transform_test, download=True)

## 課題
1. ネットワーク構造を変えて実験しましょう． 
     * まず，1層目の畳み込み層のフィルタ数を32にしましょう．また，2層目の畳み込み層のフィルタ数を64にしましょう．
    * 次に，中間層のユニット数を2048にしましょう．
   


2. 最適化の方法をAdamに変えて実験しましょう．



3. エポック数やミニバッチサイズを変えて実験しましょう．
    * まず，ミニバッチサイズを128にしましょう．
    * 次に，エポック数を50にしましょう．